In [1]:
import math
from tensorboardX import SummaryWriter
import roboschool
sess = None

In [2]:
import tensorflow as tf
import collections
gpu_options = tf.GPUOptions(allow_growth=True,per_process_gpu_memory_fraction=0.8)
tf.reset_default_graph()
sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

In [3]:
import gym
import wrappers
env_name = 'SuperMarioBros-v1'# 'PongNoFrameskip-v4' #SuperMarioBros-v1'
#env = wrappers.make_atari_deepmind(env_name, skip=4)
#env_name = 'CartPole-v1'#'CartPole-v1' #'RoboschoolAnt-v1' #'CarRacing-v0' #'LunarLander-v2' #'Acrobot-v1' #
#env = gym.make(env_name)
from vecenv import create_super_mario_env

env = create_super_mario_env()

observation_shape = env.observation_space.shape
n_actions = env.action_space.n
shape =(None, ) + observation_shape
print(shape)
print(n_actions)

/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


(None, 84, 84, 4)
12


In [4]:
# The typical imports
import gym
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Imports specifically so we can render outputs in Jupyter.
from JSAnimation.IPython_display import display_animation
from matplotlib import animation
from IPython.display import display


def display_frames_as_gif(frames):
    """
    Displays a list of frames as a gif, with controls
    """
    #plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi = 72)
    patch = plt.imshow(frames[0])
    plt.axis('off')

    def animate(i):
        patch.set_data(frames[i])

    anim = animation.FuncAnimation(plt.gcf(), animate, frames = len(frames), interval=50)
    display(display_animation(anim, default_mode='loop'))

In [ ]:
class EnvCreator(object):
    def __init__(self, name):
        self.name = name
    def __call__(self, ):
        return gym.make(self.name)

In [ ]:
from a2c_v2 import A2CAgent
import tr_helpers
import networks
import ray
ray.init()

a2c_config = {
    'GAMMA' : 0.99,
    'TAU' : 0.9,
    'LEARNING_RATE' : 1e-4,
    'NAME' : 'Supermario',
    'SCORE_TO_WIN' : 1000,
    'EPISODES_TO_LOG' : 20, 
    'LIVES_REWARD' : 5,
    'GRAD_NORM' : 0.5,
    'ENTROPY_COEF' : 0.001,
    'TRUNCATE_GRADS' : True,
    'ENV_NAME' : env_name,
    'PPO' : True,
    'E_CLIP' : 0.2,
    'NUM_ACTORS' : 8,
    'STEPS_NUM' : 50,
    'MINIBATCH_SIZE' : 50,
    'MINI_EPOCHS' : 4,
}
#self, sess, env_name, observation_shape, actions_num, config = default_config
observation_shape = env.observation_space.shape
actions_num = env.action_space.n
action_shape = env.action_space.shape
print(action_shape)
agent = A2CAgent(sess,'mario', observation_shape, actions_num, action_shape, a2c_config)
#agent.restore('nn/a2cSuperMarioBros-v1')
agent.train()

2019-05-04 20:43:53,589	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-04_20-43-53_23838/logs.
2019-05-04 20:43:53,708	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:46378 to respond...
2019-05-04 20:43:53,870	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:54617 to respond...
2019-05-04 20:43:53,878	INFO services.py:804 -- Starting Redis shard with 6.74 GB max memory.
2019-05-04 20:43:53,956	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-04_20-43-53_23838/logs.
2019-05-04 20:43:53,958	INFO services.py:1427 -- Starting the Plasma object store with 10.11 GB memory using /dev/shm.


()
(pid=23915) /home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
(pid=23915)   result = entry_point.load(False)
(pid=23911) /home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
(pid=23911)   result = entry_point.load(False)
(pid=23917) /home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
(pid=23917)   result = entry_point.load(False)
(pid=23909) /home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
(pid=23909)   result = entry

In [ ]:
agent.save('nn/a2cSuperMarioBros-v1')
def evaluate(env,t_max=10000):
    rewards = []
    env._max_episode_steps = 9999
    print('reset')
    #env = env.old_env
    s = env.reset()
    reward = 0
    for it in range(t_max):
        action = agent.get_action(s, False)
        ad = agent.get_action_distribution([s])
        #print(ad)
        s, r, done, _ = env.step(action)
        reward += r
        
            
        if done:
            break       
        
    return reward

import gym.wrappers

#env_monitor = wrappers.make_atari_deepmind(env_name, noop_max=30, skip=4)
#env_monitor = wrappers.ReallyDoneWrapper(env_monitor)
env_monitor = gym.wrappers.Monitor(env,directory='video_ppo',force=True)

sessions = [print('reward:', evaluate(env_monitor)) for _ in range(1)]
env_monitor.close()

In [ ]:
gym.envs.registry.all()